In [18]:
import pickle
import os
import glob
import random
import numpy as np

# data_path 폴더 안에 있는 모든 .pkl 파일 경로를 리스트로 가져오기
data_folder_path = 'C:/Users/etri/Desktop/etri/classification/data/'
file_paths = glob.glob(os.path.join(data_folder_path, '*.pkl'))

# train test 분류
random.shuffle(file_paths)
num_files = len(file_paths)
num_test = int(0.5 * num_files)
test_file_paths = file_paths[:num_test]
train_file_paths = file_paths[num_test:]
print(test_file_paths)
print(train_file_paths)

train_data = []
test_data = []
for file_path in train_file_paths:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        train_data.extend(data)
for file_path in test_file_paths:
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        test_data.extend(data)
for i, (rotations, (start, end), (types)) in enumerate(data):
        rotations_array = np.array(rotations)
print(len(train_data))
print(len(test_data))

['C:/Users/etri/Desktop/etri/classification/data\\trn_2023_v0_005_interloctr.pkl']
['C:/Users/etri/Desktop/etri/classification/data\\trn_2023_v0_000_interloctr.pkl', 'C:/Users/etri/Desktop/etri/classification/data\\trn_2023_v0_005_main-agent.pkl']
248
119


In [24]:
# x_train, y_train 나누기
x_train = []
y_train = []

for entry in train_data:
    x_train.append(list(entry[0]))
    y_train.append(entry[2])

x_train = np.array(x_train)
y_train = np.array(y_train)

# x_train를 2차원으로 reshape
num_samples = x_train.shape[0]
num_features = np.prod(x_train.shape[1:])
x_train = x_train.reshape(num_samples, num_features)

# 백채널 존재시 1 존재 안하면 0
y_train = np.where(np.isin(y_train, [0, 1, 2, 3]), 1, 0)

# x_test, y_test 나누기
x_test = []
y_test = []

for entry in test_data:
    x_test.append(list(entry[0]))
    y_test.append(entry[2])

x_test = np.array(x_test)
y_test = np.array(y_test)

# x_test를 2차원으로 reshape
num_samples = x_test.shape[0]
num_features = np.prod(x_test.shape[1:])
x_test = x_test.reshape(num_samples, num_features)

# 백채널 존재시 1 존재 안하면 0
y_test = np.where(np.isin(y_test, [0, 1, 2, 3]), 1, 0)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(248, 9960)
(248,)
(119, 9960)
(119,)


In [26]:
# 다운샘플링으로 데이터 밸런싱 
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

rus = RandomUnderSampler(sampling_strategy=0.5, random_state=0)
x_train, y_train = rus.fit_resample(x_train, y_train)

print(x_train.shape)
#print(x_train_re.shape)
print(Counter(y_train))
#print(Counter(y_train_re))

(78, 9960)
Counter({np.int64(0): 52, np.int64(1): 26})


In [27]:
#모델 학습하고 예측결과와 정확도 반환하는 함수 정의
from sklearn.metrics import accuracy_score

def pred_accuracy(x_train, x_test, y_train, y_test, clf):
    clf.fit(x_train, y_train)
    pred=clf.predict(x_test)
    accuracy=accuracy_score(y_test, pred)
    return pred, accuracy

In [28]:
# decisiontree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

dt_clf=DecisionTreeClassifier()
dt_pred, dt_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, dt_clf)

print(dt_accuracy)

0.8319327731092437


In [29]:
# RandomForestClassifier 
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_pred, rf_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, rf_clf)

print(rf_accuracy)

0.8319327731092437


In [30]:
# knn 모델
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_pred, knn_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, knn_clf)

print(knn_accuracy)

0.7058823529411765


In [31]:
# adaboost
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier()
ada_pred, ada_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, ada_clf)

print(ada_accuracy)

c:\Users\etri\miniforge3\envs\etri\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.7647058823529411


In [32]:
# gradientboost
from sklearn.ensemble import GradientBoostingClassifier

gbm_clf = GradientBoostingClassifier()
gbm_pred, gbm_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, gbm_clf)

print(gbm_accuracy)

0.8235294117647058


In [33]:
# xgboost
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()
xgb_pred, xgb_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, xgb_clf)

print(xgb_accuracy)

0.8319327731092437


In [34]:
# lgbm
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier()
lgbm_pred, lgbm_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, lgbm_clf)

print(lgbm_accuracy)

[LightGBM] [Info] Number of positive: 26, number of negative: 52
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030891 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 190769
[LightGBM] [Info] Number of data points in the train set: 78, number of used features: 9030
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [35]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_pred, lr_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, lr_clf)

print(lr_accuracy)

0.25210084033613445


c:\Users\etri\miniforge3\envs\etri\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
# naive_bayes
from sklearn.naive_bayes import GaussianNB

nb_clf=GaussianNB()
nb_pred, nb_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, nb_clf)

print(nb_accuracy)

0.8319327731092437


In [37]:
# voting
from sklearn.ensemble import VotingClassifier

vo_clf = VotingClassifier(estimators=[('knn_clf',knn_clf), ('dt_clf', dt_clf), ('rf_clf', rf_clf), ('ada_clf', ada_clf), ('gbm_clf',gbm_clf), ('nb_clf', nb_clf),('lr_clf', lr_clf)], voting='soft')
vo_pred, vo_accuracy = pred_accuracy(x_train, x_test, y_train, y_test, vo_clf)

print(vo_accuracy)

c:\Users\etri\miniforge3\envs\etri\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\etri\miniforge3\envs\etri\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8319327731092437
